In [8]:
# >%matplotlib notebook

In [9]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from matminer.featurizers.structure import XRDPowderPattern
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [13, 8]

In [10]:
df = pd.read_pickle(r'C:\Python\Projects\crystal-phase-prediction\pkl_files\neutron_hfo2.pkl')
df

AttributeError: 'DataFrame' object has no attribute '_data'

In [3]:
X = df

In [4]:
labels_true = pd.read_pickle(r'C:\Python\Projects\crystal-phase-prediction\data_labels\labels_hfo2.pkl')
labels_true = labels_true.labels_0_4
labels_true

0             m
1             o
2           p-o
3             t
4             m
         ...   
1720          t
1721          m
1722          o
1723    unknown
1724          t
Name: labels_0_4, Length: 1725, dtype: object

In [5]:
# labels_true = labels_true['new_labels']
# labels_true = labels_true.values
# labels_true = labels_true.astype(int)

In [6]:
# labels_true = labels_true.replace({'m': 0, 'p-o': 1, 'o': 2, 't': 3, 'unknown': 4})
# labels_true

In [7]:
# not good
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X = scaler.fit_transform(df)
X

RecursionError: maximum recursion depth exceeded while calling a Python object

In [ ]:
wcss=[]

#we always assume the max number of cluster would be 10
#you can judge the number of clusters by doing averaging
###Static code to get max no of clusters

for i in range(1,11):
    kmeans = KMeans(n_clusters= i, init='k-means++', random_state=0)
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)

    #inertia_ is the formula used to segregate the data points into clusters

In [ ]:
plt.plot(range(1,11), wcss)
plt.title('The Elbow Method')
plt.xlabel('number of clusters')
plt.ylabel('wcss')
plt.show()

In [ ]:
from yellowbrick.cluster import KElbowVisualizer
kmeans = KMeans()

visualizer = KElbowVisualizer(kmeans, k=(2,12), timings=False)

visualizer.fit(X)        # Fit the data to the visualizer
visualizer.show()        # Finalize and render the figure

In [ ]:
mPCA = PCA(n_components=100)
PrincipleComponents = mPCA.fit_transform(X)
variance = mPCA.explained_variance_ratio_
variance_ratio = np.cumsum(np.round(variance, decimals=3)*100)
variance_ratio

In [ ]:
plt.title("PCA components VS percentage of variance")
plt.ylabel("Percentage (%)")
plt.xlabel("# of components")
plt.plot(variance_ratio)
plt.show()

In [ ]:
pca = PCA(n_components=3)
PCA = pca.fit_transform(X)

In [ ]:
kmeans = KMeans(n_clusters=5)
PCA_labels = kmeans.fit_predict(PCA)

In [ ]:
from sklearn import metrics
metrics.silhouette_score(PCA, PCA_labels, metric='l2')

In [ ]:
dfPCA = pd.DataFrame(PCA)
dfPCA['cluster'] = PCA_labels
dfPCA['labels'] = labels_true
# dfPCA.columns = ['x1','x2','x3','x4','x5','cluster', 'labels']
dfPCA

In [ ]:
dfPCA.head(50)

In [ ]:
# look at the above table
dfPCA = dfPCA.replace({'m': 1, 'p-o': 4, 'o': 3, 't': 2, 'unknown': 0})

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(dfPCA['labels'], dfPCA['cluster'])

In [ ]:
dfPCA2 = dfPCA.iloc[:, [0, 1]].copy()
dfPCA2 = dfPCA2.rename(columns={0: 'PC1', 1: 'PC2'})
dfPCA2['cluster'] = dfPCA[['cluster']]
lm = sns.lmplot(data=dfPCA2, x='PC1', y='PC2',hue='cluster', fit_reg=False, legend=True, legend_out=True)
# Access the figure
fig = lm.fig 
fig.suptitle("2D PCA Predicted Clusters (73.5% Varianz)", fontsize=12)

In [ ]:
dfPCA2 = dfPCA.iloc[:, [0, 1]].copy()
dfPCA2 = dfPCA2.rename(columns={0: 'PC1', 1: 'PC2'})
dfPCA2['labels'] = dfPCA[['labels']]
lm = sns.lmplot(data=dfPCA2, x='PC1', y='PC2',hue='labels', fit_reg=False, legend=True, legend_out=True)
# Access the figure
fig = lm.fig 
fig.suptitle("2D PCA Ground Truth Clusters (73.5% Varianz)", fontsize=12)

In [ ]:
import plotly.express as px
dfPCA3 = dfPCA.iloc[:, [0, 1, 2]].copy()
dfPCA3 = dfPCA3.rename(columns={0: 'x1', 1: 'x2', 2:'x3'})
dfPCA3['cluster'] = dfPCA[['cluster']]
fig = px.scatter_3d(
    dfPCA3, x='x1', y='x2', z='x3', color=dfPCA3['cluster'],
    labels={'x1': 'PC 1', 'x2': 'PC 2', 'x3': 'PC 3'}, title='PCA 3D K-means, XRD_low_resolution'
)
fig.show()

In [ ]:
import plotly.express as px
dfPCA3 = dfPCA.iloc[:, [0, 1, 2]].copy()
dfPCA3 = dfPCA3.rename(columns={0: 'x1', 1: 'x2', 2:'x3'})
dfPCA3['labels'] = dfPCA[['labels']]
fig = px.scatter_3d(
    dfPCA3, x='x1', y='x2', z='x3', color=dfPCA3['labels'],
    labels={'x1': 'PC 1', 'x2': 'PC 2', 'x3': 'PC 3'}, title='PCA 3D Ground Truth, XRD_low_resolution'
)
fig.show()

In [ ]:
# NMF Test 
from sklearn import decomposition, datasets, model_selection, preprocessing, metrics

In [ ]:
def get_score(model, data, scorer=metrics.explained_variance_score):
    """ Estimate performance of the model on the data """
    prediction = model.inverse_transform(model.transform(data))
    return scorer(data, prediction)

In [ ]:
ks = [3,4,6,12, 20]
perfs = []
for k in ks:
    nmf = decomposition.NMF(n_components=k, random_state=0, max_iter = 5000).fit(X)
    perfs.append(get_score(nmf, X))
print(perfs)


In [ ]:
from sklearn.decomposition import NMF

model = NMF(n_components=4, init='random', random_state=0, max_iter = 5000)
nmf_features_W = model.fit_transform(X)
nmf_componentes_H = model.components_

In [ ]:
nmf_df = pd.DataFrame(nmf_componentes_H.T)
W_df = pd.DataFrame(nmf_features_W)
W_df # weights represent abundence of phase at a given nominal composition


In [ ]:
kmeans = KMeans(n_clusters=5)
nmf_labels = kmeans.fit_predict(W_df)

In [ ]:
from sklearn import metrics
metrics.silhouette_score(W_df, nmf_labels, metric='l2')

In [ ]:
W_df['cluster'] = nmf_labels
W_df['labels'] = labels_true
W_df.columns = ['x1','x2','x3','x4','cluster', 'labels']
W_df.head(40)

In [ ]:
W_df = W_df.replace({'m': 1, 'p-o': 4, 'o': 0, 't': 3, 'unknown': 2})

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(W_df['labels'], W_df['cluster'])

In [ ]:
W_df2D  = W_df[['x1', 'x2']]
W_df2D = W_df2D.rename(columns={'x1': 'C1', 'x2': 'C2'})
W_df2D['cluster'] = W_df[['cluster']]
lm = sns.lmplot(data=W_df2D, x='C1', y='C2',hue='cluster', fit_reg=False, legend=True, legend_out=True)
# Access the figure
fig = lm.fig 
fig.suptitle("2D NMF Predicted Clusters (24.1% Varianz)", fontsize=12)

In [ ]:
W_df2D  = W_df[['x1', 'x2']]
W_df2D = W_df2D.rename(columns={'x1': 'C1', 'x2': 'C2'})
W_df2D['labels'] = W_df[['labels']]
lm = sns.lmplot(data=W_df2D, x='C1', y='C2',hue='labels', fit_reg=False, legend=True, legend_out=True)
# Access the figure
fig = lm.fig 
fig.suptitle("2D NMF Ground Truth Clusters (24.1% Varianz)", fontsize=12)

In [ ]:
import plotly.express as px
fig = px.scatter_3d(
    W_df, x='x1', y='x2', z='x3', color=W_df['labels'],
    labels={'x1': 'C1', 'x2': 'C2', 'x3': 'C3'}, title='NMF 3D Ground Truth'
)
fig.show()

In [ ]:
import plotly.express as px
fig = px.scatter_3d(
    W_df, x='x1', y='x2', z='x3', color=W_df['cluster'],
    labels={'x1': 'C1', 'x2': 'C2', 'x3': 'C3'}, title='NMF 3D K-Means Prediction'
)
fig.show()

In [ ]:
from sklearn.manifold import TSNE

Xtsne = TSNE(n_components=3, perplexity=35).fit_transform(X)
dftsne = pd.DataFrame(Xtsne)

dftsne['labels'] = labels_true
dftsne.columns = ['x1','x2', 'x3', 'labels']
dftsne

In [ ]:
kmeans = KMeans(n_clusters=5)
tsne_labels = kmeans.fit_predict(dftsne[['x1', 'x2', 'x3']])
dftsne['cluster'] = tsne_labels
dftsne.columns = ['x1','x2','x3','labels', 'cluster']
dftsne

In [ ]:
from sklearn import metrics
metrics.silhouette_score(dftsne[['x1', 'x2', 'x3']], tsne_labels, metric='l2')

In [ ]:
lm = sns.lmplot(data=dftsne, x='x1', y='x2',hue='cluster', fit_reg=False, legend=True, legend_out=True)
# Access the figure
fig = lm.fig 
fig.suptitle("2D TSNE K-Means Prediction Clusters", fontsize=12)

In [ ]:
lm = sns.lmplot(data=dftsne, x='x1', y='x2',hue='labels', fit_reg=False, legend=True, legend_out=True)
# Access the figure
fig = lm.fig 
fig.suptitle("2D TSNE Ground Truth", fontsize=12)

In [ ]:
import plotly.express as px

fig = px.scatter_3d(
    dftsne, x='x1', y='x2', z='x3', color=dftsne['labels'], title='TSNE 3D Ground Truth'
)
fig.show()

In [ ]:
dftsne.head(40)

In [ ]:
dftsne = dftsne.replace({'m': 2, 'p-o': 4, 'o': 3, 't': 0, 'unknown': 1})

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(dftsne['labels'], dftsne['cluster'])